In [7]:
import math
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import load_model
import csv
from keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import plot_model
from keras import optimizers

In [28]:
# Changeable parameter
train_dir = 'D:\capstone\dataset\similar_10calss\FGVC_similar_10class_200_train\CV_0'
val_dir = 'D:\capstone\dataset\similar_10calss\FGVC_similar_10class_200_validation\CV_0'
batch_size = 64
epoch_num = 12
load_prev_model = False
NB_IV3_LAYERS_TO_FREEZE = 249  # number of freezing layer
# Make sure to change this to the designated model
# prev_model_path = r'test_mini_4_epoch.h5'
model_metrics_path = r'InceptionV3_similar_10class_200-NB_IV3_LAYERS-history.csv'

In [3]:
# Non-changeable parameter
cur_model_path = r'D:\capstone\model_save\InceptionV3_similar_10class_200_NB_IV3_LAYERS.h5'

In [4]:
#Limiting the number of resources used, hopefully this works
#It kinda does, CPU usage is only around 40-50% ish
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
set_session(tf.Session(config=config))

In [5]:
# Default configuration from
# https://keras.io/preprocessing/image/
# With a little bit of change in parameter
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.


In [19]:
if load_prev_model == False:
    # Just to make sure that this is only being loaded once
    base_model = InceptionV3(include_top=False,weights='imagenet',input_shape=(299,299,3))

    x = base_model.output
    # Output shape is (1,1,2048) no need for GlobalAveragePooling
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freezing all the base model layers
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
         layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
         layer.trainable = True

    # and then compile the model
    adam=optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
else:
    model = load_model(prev_model_path)

In [16]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_3
1 conv2d_189
2 batch_normalization_189
3 activation_189
4 conv2d_190
5 batch_normalization_190
6 activation_190
7 conv2d_191
8 batch_normalization_191
9 activation_191
10 max_pooling2d_9
11 conv2d_192
12 batch_normalization_192
13 activation_192
14 conv2d_193
15 batch_normalization_193
16 activation_193
17 max_pooling2d_10
18 conv2d_197
19 batch_normalization_197
20 activation_197
21 conv2d_195
22 conv2d_198
23 batch_normalization_195
24 batch_normalization_198
25 activation_195
26 activation_198
27 average_pooling2d_19
28 conv2d_194
29 conv2d_196
30 conv2d_199
31 conv2d_200
32 batch_normalization_194
33 batch_normalization_196
34 batch_normalization_199
35 batch_normalization_200
36 activation_194
37 activation_196
38 activation_199
39 activation_200
40 mixed0
41 conv2d_204
42 batch_normalization_204
43 activation_204
44 conv2d_202
45 conv2d_205
46 batch_normalization_202
47 batch_normalization_205
48 activation_202
49 activation_205
50 average_pooling2d_20
51 conv2d_201
52 

In [20]:
# TRAINING CELL
num_train_samples = train_generator.samples
train_epoch_steps = math.ceil(num_train_samples / batch_size)
num_val_samples = validation_generator.samples
val_epoch_steps = math.ceil(num_val_samples / batch_size)
train_history = model.fit_generator(train_generator,
                    steps_per_epoch=train_epoch_steps,
                    epochs=epoch_num,
                    validation_data=validation_generator,
                    validation_steps=val_epoch_steps)

Epoch 1/12
25/25 [==============================] - 61s 2s/step - loss: 2.3282 - acc: 0.1194 - val_loss: 2.2591 - val_acc: 0.2050
Epoch 2/12
25/25 [==============================] - 63s 3s/step - loss: 2.1937 - acc: 0.2319 - val_loss: 2.1822 - val_acc: 0.2525
Epoch 3/12
25/25 [==============================] - 65s 3s/step - loss: 2.0846 - acc: 0.3325 - val_loss: 2.1055 - val_acc: 0.3075
Epoch 4/12
25/25 [==============================] - 72s 3s/step - loss: 1.9887 - acc: 0.3850 - val_loss: 2.0263 - val_acc: 0.3550
Epoch 5/12
25/25 [==============================] - 64s 3s/step - loss: 1.8719 - acc: 0.4744 - val_loss: 1.9430 - val_acc: 0.3875
Epoch 6/12
25/25 [==============================] - 64s 3s/step - loss: 1.7643 - acc: 0.5225 - val_loss: 1.8513 - val_acc: 0.4200
Epoch 7/12
25/25 [==============================] - 65s 3s/step - loss: 1.6594 - acc: 0.5400 - val_loss: 1.7695 - val_acc: 0.4575
Epoch 8/12
25/25 [==============================] - 59s 2s/step - loss: 1.5603 - acc: 0.57

In [ ]:
#Numbering the current epoch based on the previous epoch path
#Handling multiple re-run of the training cell as well
# split_prev_model_path = prev_model_path.split('_') if cur_model_path is None else cur_model_path.split('_')
# prev_epoch_number = int(split_prev_model_path[2])
# cur_epoch_number = prev_epoch_number + epoch_num
# cur_model_path = '_'.join(split_prev_model_path[:2]+[str(cur_epoch_number)]+split_prev_model_path[3:])

In [29]:
# Record the train history in the designated csv file
# Note: newline has to be '' due to the way csv writerow works
with open(model_metrics_path, 'a+', newline='') as history_file:
    csv_writer = csv.writer(history_file)
    for e in train_history.epoch:
        epoch_number = e
        csv_writer.writerow([epoch_number,
                            train_history.history['acc'][e],
                            train_history.history['loss'][e],
                            train_history.history['val_acc'][e],
                            train_history.history['val_loss'][e]])

In [27]:
# Optionally save the model
#model.save(prev_model_path)
model.save(cur_model_path)